In [ ]:
%load_ext autoreload
%autoreload 2

# Zgoubidoo tutorial

A gentle introduction to Zgoubidoo: a modern Python 3 interface to the Zgoubi ray-tracing code.

Tutorial's objectives:
- Build Zgoubi simulations from scratch with no advanced knowledge of Zgoubi or Python
- Introduce key concepts usefull for a wide range of Zgoubi simulations
- Highlights advantages of Zgoubidoo: ease of use, repeatability, speed (especially on multi-core computers, aka. any computers)

## Getting started

Assume you have the `zgoubi` executable located somewhere in your path.

### Import zgoubidoo

In [ ]:
import zgoubidoo
from zgoubidoo.commands import *

All physical quantities used by `zgoubidoo` have units. For simplicity the 'units registry' can get a short name:

In [ ]:
from zgoubidoo import ureg as _


Let's have a first look at units:

In [ ]:
a = 1 * _.m
b = 1 * _.m + 10 * _.cm
b += a
b.to('hectometers').magnitude

A bit more interesting:

In [ ]:
brho = 1 * _.tesla * 10 * _.cm
brho.to('kilogauss meter')

**Exercice**: define quantities in other units of interest for Zgoubi simulations. In particular, try angles, energies, etc.

### Import additionnal very useful Python packages

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

**Exercice**: Have a look at what `matplotlib`, `numpy` and `pandas` are and what they can do.

### A first look at Zgoubi keywords and Zgoubidoo commands

All `zgoubi` commands ('KEYWORDS') have their equivalent `zgoubidoo` class. They all inhereit from the `zgoubidoo.commands.Command` base class which overloads a bunch of "special" Python functions. Most notably the `__str__` function.

In [ ]:
Quadrupole

In [ ]:
Venus

In [ ]:
ChangRef

In [ ]:
Fit

In [ ]:
Particule

In [ ]:
Tosca

**Exercice**: Find your favorite command. Complain if it's not available. (*bonus*) Implement a new command, just to see how easy it is.

And so on.

To create a command one needs to instanciate an object from the corresponding class:

In [ ]:
Quadrupole('Q1')

All commands have sensible (at least I'm aiming for that) default values. This means that it is possible to create a working command without changing anything.

Note that the *LABEL1* attribute gets an automatically created random value. This is useful, see later.

Of course the next step is to look at the parameters of the command. The `PARAMETERS` attribute of each class provides the full list of available parameters, along with their default values and a documentation string.

In [ ]:
Quadrupole.PARAMETERS

When the command is created those parameters are instanciated as `attributes`:

In [ ]:
Quadrupole(XL=10 * _.cm).attributes

Some have default values:

In [ ]:
Quadrupole().defaults
# Default value for R0 ==> set it to 10 cm

Others have non-default values:

In [ ]:
Quadrupole()

In [ ]:
Quadrupole().nondefaults

Note that the initializer of `Quadrupole` is taking the initiative of setting `XE` and `XS` to twice the bore radius.

Specific values for the attributes can be defined at the creation of the object:

In [ ]:
Quadrupole('COOL_QUAD', XL=777*_.kilometers, XPAS=30*_.mm)

Of course it is possible to change and access the attributes values at any time:

In [ ]:
my_quad = Quadrupole('COOL_QUAD', XL=2*_.m, XPAS=3*_.centimeter)
my_quad.KPOS = 2
my_quad.B0 = 2 * _.tesla
print(f"The field at the pole tips is now {my_quad.B0}.")
my_quad

In [ ]:
my_quad.XL

And look again at the nondefault attributes:

In [ ]:
my_quad.nondefaults

**Exercice**: define other objects, explore the different Zgoubi keywords, change the attributes, etc. In particular: create a custom BEND, a custome DRIFT.

What if your favorite command is missing?

Zgoubidoo provides different mechanisms to deal with that, maybe the easiest one is to use the `Fake` command:

In [ ]:
Fake

In [ ]:
Fake('FAKE1', INPUT="""
    'FRANCOIS' {LABEL1} 
    1.0 2.0 3.0
"""
)

In [ ]:
class FancyQuadrupole(Quadrupole):
    PARAMETERS = {
        'B0': 77 * _.tesla,
        'C0_E': 33.3,
    }
    
    def __post__init__(arg1):
        pass

my_quad = FancyQuadrupole('MY_FANCYQ')
print(my_quad.B0)
my_quad

### Creating a first Zgoubi input file

To create a zgoubi input file `zgoubidoo` provides a dedicated class: `zgoubidoo.Input`. An input can have a name and will hold a list of `zgoubidoo.commands` objects.

The `zgoubidoo.Input` objects also override the `__str__` method, which allows to automatically print them or save them to files.

In [ ]:
zgoubidoo.Input()

In [ ]:
zgoubidoo.Input(name='MY_LINE')

To make life easier, `zgoubidoo` will automatically add a `End` command at the end of each input.

Let's get started and create a FODO sequence.

In [ ]:
qf = Quadrupole('QF', XL=1*_.m, B0=1 * _.tesla)
qd = Quadrupole('QD', XL=1*_.m, B0=-1 * _.tesla)

zi = zgoubidoo.Input(name='FODO', line=[
    qf,
    Drift(XL=1 * _.m),
    qd,
    Drift(XL=1 * _.m),
])  # zi stands for `zgoubi input`
zi

We are close to being able to run this with Zgoubi. But one more thing: let's define the particle type and add a beam (*ie.* a `zgoubi` objet).

Zgoubidoo defines classes for a relatively large set of common particles:

In [ ]:
import inspect
inspect.getmembers(zgoubidoo.commands.particules, inspect.isclass)

Let's use a proton:

In [ ]:
Electron()

In [ ]:
Proton()

We will also need to define the energy, momentum, etc. of the particles. To that end, `zgoubidoo` provides a very easy to use `Kinematics` class:

In [ ]:
k = zgoubidoo.Kinematics(2 * _.GeV)
k

The constructor will infer the quantity based on the units, and provide a bunch of conversions if needed:

In [ ]:
k.brho.to('kilogauss centimeter')

Next step is to define a `zgoubi` *Objet*. We can either directly use the objet classes or use an abstraction provided by `zgoubidoo`: `Beam`.
    
Let's start with `Objet2`.

In [ ]:
Objet2('BUNCH', BORO=k.brho)

We are all set, let's redfine the `zgoubi.Input`:

In [ ]:
qf = Quadrupole('QF', XL=1*_.m, B0=1 * _.tesla)
qd = Quadrupole('QD', XL=1*_.m, B0=-1 * _.tesla)

zi = zgoubidoo.Input(name='FODO', line=[
    Objet2('BUNCH', BORO=k.brho),
    Proton(),
    qf,
    Drift(XL=1 * _.m),
    qd,
    Drift(XL=1 * _.m),
])  # zi stands for `zgoubi input`
zi.IL = 2
zi;

At this point you should be really impatient to run this with Zgoubi...

As you guessed `zgoubidoo` provides a class `Zgoubi` which is an abstraction to the `zgoubi` executable:

In [ ]:
z = zgoubidoo.Zgoubi()
z

It doesn't do much, but when you **call** it `zgoubi` will be run:

In [ ]:
z(zi)

Looked like nothing happened... but `zgoubi` has been executed.

Now a little detour: `zgoubidoo` works beautifully with multi-threading, all in a transparent way. This means that you can launch multiple instances of `zgoubi` at the same time, even in an interactive session like this one, without blocking.

The drawback is that you need to collect the results:

In [ ]:
z = zgoubidoo.Zgoubi()

z(zi)

zr = z.collect()

z.cleanup()

In [ ]:
type(zr)

In [ ]:
z.cleanup()
zr = z(zi).collect()  # zr stands for 'zgoubi results'
zr

Once more, all the resutls are encapsulated in a `ZgoubiResults` class.

It has many functionalities:

In [ ]:
zr.paths

In [ ]:
!ls /var/folders/r0/hjx4gm291nlgl0mk9wm703tm0000gn/T/tmpu5x_wswu

In [ ]:
zr.print()

The default behavior of `ZgoubiResults.print()` is to show the `zgoubi.res` file.

It is possible to look at other results.

In [ ]:
zr.print('stdout')

OK, now this is really cool! Look at this:

In [ ]:
zi[Drift][0].LABEL1

In [ ]:
zr.tracks.query("LABEL1 == '63fcb8984e0040b5b602'")

The tracking results have been automatically extracted and collected in a nice look `pandas.DataFrame`.

The other typicall "results files" of `zgoubi` can be read as well:

In [ ]:
# ! read in zgoubi/*.f grep PRINT  for all the supported PRINT commands

In [ ]:
zr.srloss

In [ ]:
zr.matrix

In [ ]:
zr.optics

But of course we'll have to work a bit harder for that.

Let's try to get some Twiss parameters.

Back to the input.

The line is a list (actually a `deque`) of commands. Let's see how we can manipulate it.

In [ ]:
# ! zpop can read in .fai while zgoubi is running (for a long run). look at what's possible to do
# FORTRAN flush command

In [ ]:
len(zi)  # 6 commands at this point

In [ ]:
zi.line.append(Matrix())

In [ ]:
zi.QD

In [ ]:
list(filter(lambda _: _.LABEL1 == 'QD', zi))


In [ ]:
# !! Could we access the commands using the LABEL2 ? Look at what can be done

In [ ]:
zi.replace('BUNCH', Objet5('BUNCH', BORO=k.brho))

This worked as expected. Let's run it again.

In [ ]:
z.cleanup()
zr = z(zi).collect()

We would like to see if this worked, but without scrolling through the entire output...

In [ ]:
# BUGG zi.zgoubi_index('QD')

In [ ]:
print('\n'.join(zi[Matrix][0].output[0][1]))

In [ ]:
zr.matrix#.at[0, 'R11']

**Exercice**: go back and adapt the FODO example to obtain a stable solution.

### Visualization with Zgoubidoo

Zgoubidoo offers default visualization to plot beamlines, tracking data, etc. Let's have a look.

In [ ]:
zi.plot()

OK... what is this "survey" thing?

All elements of a Zgoubidoo beamline are positionned with respect to a given coordinates frame. The survey operation consists in associating a global reference frame with the beamline.

In [ ]:
zi.survey()

In [ ]:
10 * zi.QD.entry.x + 2 * _.m

Let's make some nice plots...

In [ ]:
artist = zgoubidoo.vis.ZgoubiMpl()
zgoubidoo.vis.cartouche(line=zi, artist=artist)
zi.plot(ax=artist.ax, tracks=zr.tracks)

Let's modify the input to improve the plotting. The following example will:

 - use the `FakeDrift` command to force the continuous visualization of the tracks;
 - illustrate how to add more elements to the input;
 - illustrate how one can misalign elements.

In [ ]:
# There is an option to have many steps in drift.... LOOK AT THE MANUAL CEDRIC :) 

# Change the default behavior for MARKER ==> .plt ==> default = False

In [ ]:
Marker(with_plt=False)

In [ ]:
qf = Quadrupole('QF', XL=1*_.m, B0=0.1 * _.tesla, ALE=5*_.degree, KPOS=1)
qd = Quadrupole('QD', XL=1*_.m, B0=-0.05 * _.tesla, YCE=10*_.cm, KPOS=2)

zi = zgoubidoo.Input(name='FODO', line=[
    Objet2('BUNCH', BORO=k.brho),
    Proton(),
    qf,
    FakeDrift(XL=1 * _.m),  # FakeDrift is actually a MULTIPOLE with a very small but non zero B0 field, forcing zgoubi to track
    qd,
    FakeDrift(XL=1 * _.m),  # FakeDrift is actually a MULTIPOLE with a very small but non zero B0 field, forcing zgoubi to track
    Multipole(XL=1 * _.m, B1=5 * _.kilogauss),
])
zi.survey()

#zi.XPAS = 0.01 * _.m  # Note that the parameters of all the elements in the input can be set with a single command
z = zgoubidoo.Zgoubi()
zr = z(zi).collect()

artist = zgoubidoo.vis.ZgoubiMpl()
zgoubidoo.vis.cartouche(line=zi, artist=artist)
zi.plot(ax=artist.ax, tracks=zr.tracks)

At the end of a Zgoubi run it is possible to save the input file and one or more output files to a permanent directory (by default Zgoubidoo runs everything in temporary directories).

In [ ]:
zr.save('.', ['zgoubi.dat', 'zgoubi.res', 'zgoubi.log'])

## Tracking on multi-core machines

`zgoubi` itself is purely single process, single thread. However, `zgoubidoo` makes it easy to track particles on multi-core machines. Doing so with `zgoubidoo` is totaly transparent for the user, at the time of the input preparation and at the time of the collection of the results.

To illustrate this, let's reuse our previous input, but this time with a real bunch. This is so common that `zgoubidoo` provides an abstraction layer on top of the `zgoubi` *objets*.

### Using `zgoubidoo` beams

`zgoubidoo` provides different subclasses of `zgoubidoo.commands.beam.Beam` to suit specific needs. Here we will use the `BeamZgoubiDistribution` 

In [ ]:
my_bunch = BeamZgoubiDistribution('BUNCH', kinematics=k, particle=Proton, IMAX=1e4)
my_bunch

In [ ]:
qf = Quadrupole('QF', XL=1*_.m, B0=0.1 * _.tesla, ALE=5*_.degree, KPOS=1)
qd = Quadrupole('QD', XL=1*_.m, B0=-0.05 * _.tesla, YCE=10*_.cm, KPOS=2)

zi = zgoubidoo.Input(name='FODO', line=[
    my_bunch,
    qf,
    FakeDrift(XL=1 * _.m),  # FakeDrift is actually a MULTIPOLE with a very small but non zero B0 field, forcing zgoubi to track
    qd,
    FakeDrift(XL=1 * _.m),  # FakeDrift is actually a MULTIPOLE with a very small but non zero B0 field, forcing zgoubi to track
])

#zi.XPAS = 1.0 * _.cm  # Note that the parameters of all the elements in the input can be set with a single command

Let's run `zgoubi` and track the distribution and check how long it takes.

In [ ]:
%%timeit -n 1 -r 1
z = zgoubidoo.Zgoubi()
zr = z(zi).collect()

That's about 20 seconds for 1e4 particules. The results (in this case the tracking data) are collected automatically:

In [ ]:
print(zr.tracks.shape)
zr.tracks.head(5)

OK, now let's try again but using the true power of our multi-cores machines.

To that end `zgoubidoo` introduces the concept of **slices**. A given bunch (beam) will be divided in multiple slices. Each slice will be run by its own instance of `zgoubi`, in parallel.

In [ ]:
my_bunch.slices = 4

In [ ]:
# Choice of the word 'slice'  ==> batch ?
# Actually slicing

In [ ]:
%%timeit -n 1 -r 1
z = zgoubidoo.Zgoubi()
zr = z(zi).collect()

As expected we gained almost a factor 4!

And all the results from the 4 runs are collected together:

In [ ]:
print(zr.tracks.shape)
zr.tracks.head(5)

### Parametric scans

TODO

## Importing MAD-X sequences

Zgoubidoo offers different interfaces to automatically load and convert MAD-X Twiss sequences. This section illustrates how this can be done using the LHeC as an example.

Zgoubidoo provides a `sequences` module aimed at abstracting the concepts of 'sequences' and 'beamlines'. A `zgoubidoo.sequences.Sequence` object is *not* a Zgoubi input: it is a generic sequence holding information to recreate a beamline. It is also 'code-independent': the information contained in the sequence can be used by 'converters' to convert the sequence onto a valid Zgoubi or MAD-X sequence.

The `Sequence` class provides different 'loaders' to initialize the sequence.

In [ ]:
lhec_sequence = zgoubidoo.sequences.Sequence.from_madx_twiss(
    filename='twiss.outx',
    path='/Users/chernals/Downloads'
)

The loader create a specialized `TwissSequence` object:

In [ ]:
type(lhec_sequence)

The loaders will load not only the main Twiss table, but will also read the metadata and instanciate other quantities. The main Twiss table can be used as a `pandas.DataFrame`.

In [ ]:
lhec_sequence.df.head(5)

The Twiss headers are also available in the form of a `pandas.Series`:

In [ ]:
lhec_sequence.metadata.data

A `Kinematics` object is automatically infered from the Twiss headers.

In [ ]:
lhec_sequence.metadata.kinematics

Same for the particle type.

In [ ]:
lhec_sequence.metadata.particle

Also, Zgoubidoo provides a `BetaBlock` class that holds a set of Twiss parameters. The sequence will contain the 'beta0' block:

In [ ]:
lhec_sequence.betablock

Finally, as this is a Twiss sequence, a `BeamTwiss` is also created. This is another type of `zgoubidoo.beam` that abstracts the concept of `OBJET 5` from Zgoubi.

In [ ]:
lhec_sequence.beam

Now that we have the sequence, we need to convert it to a Zgoubi input. This is equallly easy to do.

Note that the beam definition is automatically included, with the correct initial Twiss parameters, correct particle type and correct BRHO.

In [ ]:
zi_twiss = zgoubidoo.Input.from_sequence(lhec_sequence)

In [ ]:
#zi_twiss

## Real machine Twiss computations

Let's add a Zgoubi 'OPTICS' keyword and see if we can reproduce the MAD-X Twiss.

In [ ]:
#zi_twiss.XPAS = 20 * _.cm
zi_twiss.insert_after(zi_twiss.beam, Optics())
zi_twiss.survey(output=False)

In [ ]:
z = zgoubidoo.Zgoubi()
zr_twiss = z(zi_twiss).collect()

In [ ]:
fig = plt.figure(figsize=(15, 7))
ax = fig.add_subplot(111)
artist = zgoubidoo.vis.ZgoubiMpl(ax=ax)
zgoubidoo.vis.cartouche(line=zi_twiss, artist=artist)

ax.plot(lhec_ir.df['S'], lhec_ir.df['BETX'], 'b-', ms=10, label='MAD-X BETX')
ax.plot(zr_twiss.optics['S'], zr_twiss.optics['BETA11'], 'bx', ms=7, label='Zgoubi BETA11')

ax.plot(lhec_ir.df['S'], lhec_ir.df['BETY'], 'r-', ms=10, label='MAD-X BETY')
ax.plot(zr_twiss.optics['S'], zr_twiss.optics['BETA22'], 'rx', ms=7, label='Zgoubi BETA22')


ax2 = ax.twinx()
ax2.plot(lhec_ir.df['S'], lhec_ir.df['DX'], 'g-', ms=10, label='MAD-X DX')
ax2.plot(zr_twiss.optics['S'], zr_twiss.optics['DISP1'], 'gx', ms=7, label='Zgoubi DISP1')

ax2.plot(lhec_ir.df['S'], lhec_ir.df['DY'], 'm-', ms=10, label='MAD-X DY')
ax2.plot(zr_twiss.optics['S'], zr_twiss.optics['DISP3'], 'mx', ms=7, label='Zgoubi DISP3')

ax.legend(loc=6, fontsize=12)
ax2.legend(loc=5, fontsize=12)
artist.ax.grid(True, alpha=0.2)
artist.ax.set_xlabel("S (m)", fontsize=20)
artist.ax.set_ylabel("Beta function (m)", fontsize=20)
artist.ax.tick_params(axis='both', which='major', labelsize=18)
ax2.set_ylabel("Dispersion (m)", fontsize=20)
ax2.tick_params(axis='both', which='major', labelsize=18)

Encouraging but not fully correct.

The yellow-colored magnets are vertical bends, rotated using a `CHANGREF` Zgoubi keyword. The Zgoubi tracking is done in a local coordinate system for each magnet. Therefore Zgoubi does not "understand" that the axes are swapped when rotating the magnet to make it a vertical bend. As a consequence, the reconstruction of the transfer matrix, and thus the computation of the Twiss parameters becomes incorrect.

The same happens for the vertical and horizontal beta-functions.

We can also observe what happens with the alpha-functions.

In [ ]:
fig = plt.figure(figsize=(15, 7))
ax = fig.add_subplot(111)
artist = zgoubidoo.vis.ZgoubiMpl(ax=ax)
zgoubidoo.vis.cartouche(line=zi_twiss, artist=artist)

ax.plot(lhec_ir.df['S'], lhec_ir.df['ALFX'], 'b-', ms=10, label='MAD-X ALPHAX')
ax.plot(zr_twiss.optics['S'], zr_twiss.optics['BETA11'], 'bx', ms=7, label='Zgoubi ALPHA11')

ax.plot(lhec_ir.df['S'], lhec_ir.df['ALFY'], 'b-', ms=10, label='MAD-X ALPHAY')
ax.plot(zr_twiss.optics['S'], zr_twiss.optics['BETA22'], 'bx', ms=7, label='Zgoubi ALPHA22')

ax.legend(loc=3, fontsize=12)
artist.ax.grid(True, alpha=0.2)
artist.ax.set_xlabel("S (m)", fontsize=20)
artist.ax.set_ylabel("Alpha function", fontsize=20)
artist.ax.tick_params(axis='both', which='major', labelsize=18)

#ax.text(100.0, 1500.0, f"{lhec_ir.table.loc['B0', 'ALFX']}", fontsize=12)
#ax.text(100.0, 1300.0, f"""{tw.query("LABEL1 == 'IP'").iloc[-1]['ALPHA11']:.4f}""", fontsize=12)

**Not cool!**

Can we do better?

Zgoubidoo is able to compute the Twiss parameters in the same way than Zgoubi:
- the transfer matrix is computed using finite differences
- the Twiss parameters are reconstructed

However, because of the powerful `survey` module in Zgoubidoo, the location and orientation of every element is known. Therefore, when reading the tracking data, Zgoubidoo will automatically convert the coordinates to the global coordinate system (remember, that's the one that we defined when doing the survey). The angles are also converted, so in the end all the Twiss paraters will behave correctly; even for crazy lattices with arbitraty rotations.

First we need to compute the transfer matrix.

In [ ]:
zr_twiss.tracks['T'] = zr_twiss.tracks['TG']
zr_twiss.tracks['P'] = zr_twiss.tracks['PG']
zr_twiss.tracks['X'] = zr_twiss.tracks['XG']
zr_twiss.tracks['Y'] = zr_twiss.tracks['YG']
zr_twiss.tracks['Z'] = zr_twiss.tracks['ZG']
M = zgoubidoo.twiss.compute_transfer_matrix(beamline=zi_twiss, tracks=zr_twiss.tracks)
M.head(5)

Then we compute the Twiss parameters (note that the `BetaBlock` is used here):

In [ ]:
zgoubidoo_twiss = zgoubidoo.twiss.compute_twiss(matrix=M, twiss_init=lhec_sequence.betablock)
zgoubidoo_twiss.head(5)

We should now be able to plot everything.

In [ ]:
fig = plt.figure(figsize=(15, 7))
ax = fig.add_subplot(111)
artist = zgoubidoo.vis.ZgoubiMpl(ax=ax)
zgoubidoo.vis.cartouche(line=zi_twiss, artist=artist)

ax.plot(zgoubidoo_twiss['S'], zgoubidoo_twiss['BETA11'], 'b-', ms=0.5, label='Zgoubidoo BETA11')
ax.plot(lhec_ir.df['S'] + zgoubidoo_twiss['S'].min(), lhec_ir.df['BETX'], 'b+', ms=10, label='MAD-X BETX')
#ax.plot(zr_twiss.optics['S'] + tw['S'].min(), zr_twiss.optics['BETA11'], 'bx', ms=7, label='Zgoubi BETA11')

ax.plot(zgoubidoo_twiss['S'], zgoubidoo_twiss['BETA22'], 'r-', ms=0.5, label='Zgoubidoo BETA22')
ax.plot(lhec_ir.df['S'] + zgoubidoo_twiss['S'].min(), lhec_ir.df['BETY'], 'r+', ms=10, label='MAD-X BETY')
#ax.plot(zr_twiss.optics['S'] + tw['S'].min(), zr_twiss.optics['BETA22'], 'rx', ms=7, label='Zgoubi BETA22')


ax2 = ax.twinx()
ax2.plot(zgoubidoo_twiss['S'], zgoubidoo_twiss['DISP1'], 'g-.', ms=0.5, label='Zgoubidoo DISP1')
ax2.plot(lhec_ir.df['S'] + zgoubidoo_twiss['S'].min(), lhec_ir.df['DX'], 'g+', ms=10, label='MAD-X DX')
#ax2.plot(zr_twiss.optics['S'] + tw['S'].min(), zr_twiss.optics['DISP1'], 'gx', ms=7, label='Zgoubi DISP1')

ax2.plot(zgoubidoo_twiss['S'], zgoubidoo_twiss['DISP3'], 'm-.', ms=0.5, label='Zgoubidoo DISP3')
ax2.plot(lhec_ir.df['S'] + zgoubidoo_twiss['S'].min(), lhec_ir.df['DY'], 'm+', ms=10, label='MAD-X DY')
#ax2.plot(zr_twiss.optics['S'] + tw['S'].min(), zr_twiss.optics['DISP3'], 'mx', ms=7, label='Zgoubi DISP3')

ax.legend(loc=6, fontsize=12)
ax2.legend(loc=5, fontsize=12)
artist.ax.grid(True, alpha=0.2)
artist.ax.set_xlabel("S (m)", fontsize=20)
artist.ax.set_ylabel("Beta function (m)", fontsize=20)
artist.ax.tick_params(axis='both', which='major', labelsize=18)
ax2.set_ylabel("Dispersion (m)", fontsize=20)
ax2.tick_params(axis='both', which='major', labelsize=18)

## backward tracking ! For transfer matrix in case of large fringe fields, etc. check with the S coordinate

Lattices for the workshop : ESRF ? Or PSR ? Maybe PSR is easier. Then FFA.

PSR
FFA
ESRF